<a href="https://colab.research.google.com/github/Desi-Noviana/Analisis-Sentimen-Ojek-Online/blob/main/SKRIPSI_ANALISIS_SENTIMEN_OJOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install swifter

In [ ]:
pip install plotly-express

In [ ]:
pip install gensim

In [ ]:
pip install wordcloud

In [ ]:
pip install tensorflow

In [ ]:
pip install Sastrawi

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import keras
import swifter
import tensorflow as tf
import gensim
from sklearn.metrics import confusion_matrix
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize.treebank import TreebankWordDetokenizer
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop,Adam,SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df = pd.read_csv('Preprocessing_ojol.csv', error_bad_lines = False)
pd.set_option('max_colwidth', None)
df.head()

# Visualiasi Data

In [ ]:
fig = px.histogram(df, x="sentiment", title="Jumlah Sentiment")
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
cv = CountVectorizer(stop_words = 'english')
words = cv.fit_transform(df.Ulasan)

sum_words = words.sum(axis=0)

words_freq = [(word, sum_words[0, i]) for word, i in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)

frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])

frequency.head(30).plot(x='word', y='freq', kind='bar', figsize=(15, 7), color = 'red')
plt.title("Most Frequently Occuring Words - Top 30")

In [ ]:
wr = pd.read_csv('Preprocessing_ojol.csv', error_bad_lines = False)

In [ ]:
wr['sentiment'] = wr['sentiment'].replace('positive', 2)
wr['sentiment'] = wr['sentiment'].replace('negative', 0)
wr['sentiment'] = wr['sentiment'].replace('neutral', 1)

In [ ]:
wordcloud = WordCloud(background_color = 'white', width = 1000, height = 1000).generate_from_frequencies(dict(words_freq))

plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.title("WordCloud Kata Yang Muncul", fontsize = 15)

In [ ]:
neutral_words =' '.join([text for text in wr['Ulasan'][wr['sentiment'] == 1]])

wordcloud = WordCloud(background_color = 'white',width=1920, height=1080, random_state=5, max_font_size = 500).generate(neutral_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('The Neutral Words')
plt.show()

In [ ]:
positive_words =' '.join([text for text in wr['Ulasan'][wr['sentiment'] == 2]])

wordcloud = WordCloud(background_color = 'white',width=800, height=500, random_state=5, max_font_size = 110).generate(positive_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('The Positive Words')
plt.show()

In [ ]:
negative_words =' '.join([text for text in wr['Ulasan'][wr['sentiment'] == 0]])

wordcloud = WordCloud(background_color = 'white',width=800, height=500, random_state=5, max_font_size = 110).generate(negative_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('The Negative Words')
plt.show()

# Label Encoding

In [ ]:
data = pd.read_csv('Preprocessing_ojol.csv', error_bad_lines = False)

In [ ]:
ulasan = data['Ulasan']

In [ ]:
labels = np.array(data['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 'negative':
        y.append(0)
    if labels[i] == 'neutral':
        y.append(1)
    if labels[i] == 'positive':
        y.append(2)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="int64")
del y

In [ ]:
len(labels)

# Proses Data

In [ ]:
temp = []
#Splitting pd.Series to list
data_to_list = ulasan.values.tolist()
for i in range(len(data_to_list)):
    temp.append((data_to_list[i]))
list(temp[:10])

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(temp))

print(data_words[:10])

In [ ]:
len(data_words)

In [ ]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [ ]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:10])

In [ ]:
data = np.array(data)

# Mengubah Data Kedalam Bentuk Angka

In [ ]:
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
ulasan = pad_sequences(sequences, maxlen=max_len)
print(ulasan)

In [ ]:
embedding_layer = Embedding(1000, 64)

In [ ]:
print(embedding_layer)

In [ ]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(ulasan,labels,test_size=0.2,random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

print("y_train",y_train.shape)
print("y_test",y_test.shape)

In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 10))
model1.add(layers.LSTM(20,dropout = 0.5,recurrent_dropout=0.5))
model1.add(layers.Dense(3, activation='softmax'))
model1.summary()


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto',period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=30,batch_size=28,validation_data=(X_test, y_test),callbacks=[checkpoint1])

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.57,recurrent_dropout=0.57)))
model2.add(layers.Dense(3,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=40,batch_size=28,validation_data=(X_test, y_test),callbacks=[checkpoint2])

# Menyimpan Model

In [ ]:
best_model = keras.models.load_model("best_model1.hdf5")

# Evaluasi Model

In [ ]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'blue', label='Training acc')
plt.plot(epochs, val_acc, 'red', label='Validation acc')
plt.title('Percobaan 1', fontsize=15)
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'blue', label='Training loss')
plt.plot(epochs, val_loss, 'red', label='Validation loss')
plt.legend()
 
plt.show()